In [ ]:
# Install deepinv (skip if already installed)
%pip install deepinv

<!-- MathJax macro definitions inserted automatically -->
$$
\newcommand{\forw}[1]{{A\left({#1}\right)}}
\newcommand{\noise}[1]{{N\left({#1}\right)}}
\newcommand{\inverse}[1]{{R\left({#1}\right)}}
\newcommand{\inversef}[2]{{R\left({#1},{#2}\right)}}
\newcommand{\inversename}{R}
\newcommand{\reg}[1]{{g_\sigma\left({#1}\right)}}
\newcommand{\regname}{g_\sigma}
\newcommand{\sensor}[1]{{\eta\left({#1}\right)}}
\newcommand{\datafid}[2]{{f\left({#1},{#2}\right)}}
\newcommand{\datafidname}{f}
\newcommand{\distance}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\distancename}{d}
\newcommand{\denoiser}[2]{{\operatorname{D}_{{#2}}\left({#1}\right)}}
\newcommand{\denoisername}{\operatorname{D}_{\sigma}}
\newcommand{\xset}{\mathcal{X}}
\newcommand{\yset}{\mathcal{Y}}
\newcommand{\group}{\mathcal{G}}
\newcommand{\metric}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\loss}[1]{{\mathcal\left({#1}\right)}}
\newcommand{\conj}[1]{{\overline{#1}^{\top}}}
$$


# Uncertainty quantification with PnP-ULA.

This code shows you how to use sampling algorithms to quantify uncertainty of a reconstruction
from incomplete and noisy measurements.

ULA obtains samples by running the following iteration:

\begin{align}x_{k+1} = x_k +  \alpha \eta \nabla \log p_{\sigma}(x_k) + \eta \nabla \log p(y|x_k)  + \sqrt{2 \eta} z_k\end{align}

where $z_k \sim \mathcal{N}(0, I)$ is a Gaussian random variable, $\eta$ is the step size and
$\alpha$ is a parameter controlling the regularization.

The PnP-ULA method is described in the paper :footcite:t:`laumont2022bayesian`.

In [ ]:
import deepinv as dinv
from deepinv.utils.plotting import plot
import torch
from deepinv.utils import load_example

## Load image from the internet

This example uses an image of Messi.


In [ ]:
device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

x = load_example("messi.jpg", img_size=32).to(device)

## Define forward operator and noise model

This example uses inpainting as the forward operator and Gaussian noise as the noise model.


In [ ]:
sigma = 0.1  # noise level
physics = dinv.physics.Inpainting(mask=0.5, img_size=x.shape[1:], device=device)
physics.noise_model = dinv.physics.GaussianNoise(sigma=sigma)

# Set the global random seed from pytorch to ensure reproducibility of the example.
torch.manual_seed(0)

## Define the likelihood

Since the noise model is Gaussian, the negative log-likelihood is the L2 loss.

\begin{align}-\log p(y|x) \propto \frac{1}{2\sigma^2} \|y-Ax\|^2\end{align}


In [ ]:
# load Gaussian Likelihood
likelihood = dinv.optim.data_fidelity.L2(sigma=sigma)

## Define the prior

The score a distribution can be approximated using Tweedie's formula via the
[`deepinv.optim.ScorePrior`](https://deepinv.github.io/deepinv/api/stubs/deepinv.optim.ScorePrior.html) class.

\begin{align}\nabla \log p_{\sigma}(x) \approx \frac{1}{\sigma^2} \left(D(x,\sigma)-x\right)\end{align}

This example uses a pretrained DnCNN model.
From a Bayesian point of view, the score plays the role of the gradient of the
negative log prior
The hyperparameter ``sigma_denoiser`` ($sigma$) controls the strength of the prior.

In this example, we use a pretrained DnCNN model using the [`deepinv.loss.FNEJacobianSpectralNorm`](https://deepinv.github.io/deepinv/api/stubs/deepinv.loss.FNEJacobianSpectralNorm.html) loss,
which makes sure that the denoiser is firmly non-expansive (see :footcite:t:`terris2020building`), and helps to
stabilize the sampling algorithm.


In [ ]:
sigma_denoiser = 2 / 255
prior = dinv.optim.ScorePrior(
    denoiser=dinv.models.DnCNN(pretrained="download_lipschitz")
).to(device)

## Create the MCMC sampler

Here we use the Unadjusted Langevin Algorithm (ULA) to sample from the posterior defined in
[`deepinv.sampling.ULAIterator`](https://deepinv.github.io/deepinv/api/stubs/deepinv.sampling.ULAIterator.html).
The hyperparameter ``step_size`` controls the step size of the MCMC sampler,
``regularization`` controls the strength of the prior and
``iterations`` controls the number of iterations of the sampler.


In [ ]:
regularization = 0.9
step_size = 0.01 * (sigma**2)
iterations = int(5e3) if torch.cuda.is_available() else 10
params = {
    "step_size": step_size,
    "alpha": regularization,
    "sigma": sigma_denoiser,
}
f = dinv.sampling.sampling_builder(
    "ULA",
    prior=prior,
    data_fidelity=likelihood,
    max_iter=iterations,
    params_algo=params,
    thinning=1,
    verbose=True,
)

## Generate the measurement
We apply the forward model to generate the noisy measurement.


In [ ]:
y = physics(x)

## Run sampling algorithm and plot results
The sampling algorithm returns the posterior mean and variance.
We compare the posterior mean with a simple linear reconstruction.


In [ ]:
mean, var = f.sample(y, physics)

# compute linear inverse
x_lin = physics.A_adjoint(y)

# compute PSNR
print(f"Linear reconstruction PSNR: {dinv.metric.PSNR()(x, x_lin).item():.2f} dB")
print(f"Posterior mean PSNR: {dinv.metric.PSNR()(x, mean).item():.2f} dB")

# plot results
error = (mean - x).abs().sum(dim=1).unsqueeze(1)  # per pixel average abs. error
std = var.sum(dim=1).unsqueeze(1).sqrt()  # per pixel average standard dev.
imgs = [x_lin, x, mean, std / std.flatten().max(), error / error.flatten().max()]
plot(
    imgs,
    titles=["measurement", "ground truth", "post. mean", "post. std", "abs. error"],
)

:References:

> **Footbibliography**
>
>
